In [ ]:
] add CSV

In [ ]:
] add DataFrames

In [ ]:
] add DataFramesMeta

In [ ]:
] add WebDriver

In [ ]:
using CSV
using DataFrames
using DataFramesMeta
using ODBC

In [ ]:
] add ODBC

In [ ]:
bs = "bulk_bookstore_dataset.csv"

In [ ]:
df = CSV.read(read(bs), DataFrame);

In [ ]:
foreach(println, names(df))

In [ ]:
df = @chain df begin
    @rtransform $[:crap, :genre, :name] = begin
        x = split(:Breadcrumbs, "|")
        get.(Ref(x), 1:3, missing)
    end
end

In [ ]:
genresValues = unique(df[:,:genre]);

In [ ]:
genres = DataFrame()
genres.genre=genresValues;

In [ ]:
df.GenreId = [rownumber(genres[only(findall(==(row.genre), genres.genre)), :]) for row in eachrow(df)];

In [ ]:
CSV.write(".\\genres.csv", genres ; openquotechar='"', closequotechar='"', escapechar='"', quotestrings=true)

In [ ]:
foreach(println, names(df))

In [ ]:
df.rating = rand(1:5, nrow(df));

In [ ]:
df.copies = rand(1000:1000000,nrow(df));

In [ ]:
foreach(println, names(df))

In [ ]:
# df.pub = [x.match for x in match.(r"^(\w+)", df.Publisher)]
df.PublisherId.=1;

In [ ]:
df.year = [parse(Int64,x.match) for x in match.(r"(\d{4})", df.Publisher)];

In [ ]:
df.name[1:5]

In [ ]:
CSV.write(".\\bookIBSN.csv", df[:,[:ISBN, :name, :Price, :rating, :AuthorId, :GenreId, :PublisherId, :year,:copies, :des]]; openquotechar='"', closequotechar='"', escapechar='"', quotestrings=true)

In [ ]:
odf = CSV.read(read("out.csv"), DataFrame);

In [ ]:
odf

In [ ]:
df = CSV.read(read("bookIBSN.csv"), DataFrame);

In [ ]:
df

In [ ]:
df.description = odf[:,"description"]

In [ ]:
# rename!(df,:descrition => :description)

In [ ]:
df = select!(df, Not([:Url]))

In [ ]:
df

In [ ]:
noMissing = df[completecases(df),:]

In [ ]:
nrow(noMissing)

In [ ]:
disallowmissing!(noMissing)

In [ ]:
CSV.write(".\\bookIBSN.csv", noMissing);

In [ ]:
df = CSV.read(read("bookIBSN.csv"), DataFrame);

In [ ]:
df[1,:]

In [ ]:
df.pub

In [ ]:
df.ISBN = [SubString(s,3,15) for s in df.ISBN]

In [ ]:
df.ISBN = [parse(Int64,s) for s in df.ISBN]

In [ ]:
aDf

In [ ]:
df = filter(:AuthorId => i -> (i ∈ aDf.id), df)

In [ ]:
df

In [ ]:
CSV.write(".\\booksQuotes.csv", df ; openquotechar='"', closequotechar='"', escapechar='"', quotestrings=true)

In [ ]:
df.Price = [parse(Float64, SubString(s,2)) for s in df.Price]

In [ ]:
nSet = Set()

In [ ]:
for a in df.Author
    twoNames = split(a,",")
    if length(twoNames) == 2
        push!(nSet,string(twoNames[1]))
        push!(nSet,string(twoNames[2]))
    else
        push!(nSet,a)
    end
end

In [ ]:
df.AuthorId .= zero(nrow(df))

In [ ]:
length(nSet)

In [ ]:
for (i, val) in enumerate(nSet)
    # println(nrow(filter(:Author => n -> n == val, df)))
    # filter(:Author => n -> n == val, df).AuthorId .= i
    # df[:AuthorId, filter(:Author => n -> n == val, df)] .= i
    @. df.AuthorId = ifelse(df.Author == val, i, df.AuthorId)
    # println(filter(:Author => n -> n == val, df).AuthorId)
    # println(i)
end

In [ ]:
df.AuthorId

In [ ]:
df = df[completecases(df),:]
disallowmissing!(df)

In [ ]:
nrow(df)

In [ ]:
length(nSet)

In [ ]:
aDf = DataFrame()

In [ ]:
aDf.id = [i for (i, val) in enumerate(nSet)]
    
    

In [ ]:
function parseFirstName(s::String)::String
    names = split(s, " ")
    if length(names) == 3
        return names[1] * " " * names[2]
    end
    return names[1]
end
    

In [ ]:
function parseLastName(s::String)::String
    names = split(s, " ")
    if length(names) == 3
        return names[3]
    end
    if length(names) < 2
        println(s)
        return ""
    end
    return names[2]
end

In [ ]:
aDf.FirstName = [strip(parseFirstName(s)) for s in nSet]

In [ ]:
aDf.LastName =  [strip(parseLastName(s)) for s in nSet]

In [ ]:
aDf.Author = [strip(s) for s in nSet]

In [ ]:
afiles = readdir("./au", join=true)

In [ ]:
adf = CSV.read(read(afiles[1]), DataFrame)
for file in afiles[2:end]
    adf =  vcat(adf, CSV.read(read(file), DataFrame));
end    

In [ ]:
adf = adf[completecases(adf),:] 

In [ ]:
nrow(adf)

In [ ]:
adf.ISBN = string.(adf.ISBN)

In [ ]:
adf.ISBN = [parse(Int64,s) for s in adf.ISBN]

In [ ]:
adf = unique(adf, :ISBN);

In [ ]:
df = CSV.read(read("bookIBSN.csv"), DataFrame);

In [ ]:
nrow(adf)

In [ ]:
nrow(df)

In [ ]:
df.ISBN = [parse(Int64,s) for s in df.ISBN]

In [ ]:
fdf = filter(:ISBN => i -> (i ∈ adf.ISBN), df)

In [ ]:
authors = [r.Author for r in eachrow(df) if r.ISBN in adf.ISBN]

In [ ]:
aset = Set(authors)

In [ ]:
length(aset)

In [ ]:
df = filter(:Author => a -> (a ∉ aset), df)

In [ ]:
#filter(:name => n -> n == "Alice", grades_2020())

In [ ]:
CSV.write("bookIBSN.csv", df)

In [ ]:
aDf

In [ ]:
adf

In [ ]:
foreach(println, names(df))

In [ ]:
daf = leftjoin(df, adf, on=:ISBN);

In [ ]:
foreach(println, names(daf))

In [ ]:
aDf = leftjoin(aDf, daf, on=:Author, makeunique=true)

In [ ]:
aDf.PublisherId .= 1

In [ ]:
aDf = aDf[:,[:id, :FirstName, :LastName, :biography, :PublisherId]]

In [ ]:
aDf = aDf[completecases(aDf),:];
disallowmissing!(aDf);


In [ ]:
aDf = unique(aDf,:id);

In [ ]:
nrow(aDf)

In [ ]:
CSV.write(".\\authors.csv", aDf[:,[:id, :FirstName, :LastName, :biography, :PublisherId]]; openquotechar='"', closequotechar='"', escapechar='"', quotestrings=true)

In [ ]:
aDf = aDf[:,[:id, :FirstName, :LastName, :biography, :PublisherId]]

In [ ]:
aDf = aDf[completecases(aDf),:];
disallowmissing!(aDf);


In [ ]:
aDf = unique(aDf,:id);

In [ ]:
nrow(aDf)

In [ ]:
CSV.write(".\\authors.csv", aDf[:,[:id, :FirstName, :LastName, :biography, :PublisherId]]; openquotechar='"', closequotechar='"', escapechar='"', quotestrings=true)